In [137]:
!pip install langchain langchain-community langchain langchain-openai langchain-anthropic chromadb gitpython

# 코드 리뷰 LLM 서비스
---

# 1. 환경 변수 로드

In [138]:
from dotenv import load_dotenv
import os

load_dotenv()

class Config:
    OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
    ANTHROPIC_API_KEY = os.environ['ANTHROPIC_API_KEY']
    PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
    CHROMA_DB_PATH = os.environ['CHROMA_DB_PATH']

---
# 2. LLM(OpenAI, Anthropic)

In [139]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=Config.OPENAI_API_KEY)
anthropic_llm = ChatAnthropic(model_name="claude-3-5-sonnet-20241022", api_key=Config.ANTHROPIC_API_KEY)

---
# 3. Embedding(OpenAI)

In [140]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=Config.OPENAI_API_KEY)

---
# 4. VectorStore(ChromaDB)

In [141]:
from langchain.vectorstores import Chroma

vector_store = Chroma(persist_directory=Config.CHROMA_DB_PATH, embedding_function=embeddings)

---
# 5. Github repository -> Store

In [142]:
from git import Repo
import hashlib

# GitHub 레포지토리를 로컬로 클론
def clone_github_repo(repo_url, local_path):
    if os.path.exists(local_path):
        print("로컬에 이미 레포지토리가 존재합니다. 변경 사항을 pull합니다.")
        repo = Repo(local_path)
        repo.remotes.origin.pull()
    else:
        print(f"Cloning {repo_url} into {local_path}...")
        Repo.clone_from(repo_url, local_path)
    print("레포지토리 클론 또는 업데이트 완료!")

# 레포지토리 내 Python 파일 경로를 찾기
def find_python_files(repo_path):
    python_files = []
    for root, _, files in os.walk(repo_path):
        for file in files:
            if file.endswith(".py"):
                python_files.append(os.path.join(root, file))
    return python_files

# 파일의 해시값(SHA256)을 계산
def calculate_file_hash(file_path):
    sha256 = hashlib.sha256()
    with open(file_path, "rb") as f:
        while chunk := f.read(8192):
            sha256.update(chunk)
    return sha256.hexdigest()

# 파일이 업데이트되었는지 확인
def is_file_updated(file_path, vector_store):
    file_hash = calculate_file_hash(file_path)
    # Chroma에서 해당 파일의 메타데이터 검색
    results = vector_store.similarity_search(file_path, k=1, filter={"file_path": file_path})
    if results:
        # 저장된 해시값과 비교
        return results[0].metadata.get("file_hash") != file_hash
    return True

# Python 파일을 읽어 벡터화하고 Chroma DB에 저장
def store_python_files_in_chroma(file_paths, vector_store):
    for file_path in file_paths:
        try:
            if not is_file_updated(file_path, vector_store):
                print(f"파일이 변경되지 않았습니다. 건너뜁니다: {file_path}")
                continue

            with open(file_path, "r", encoding="utf-8") as file:
                code_content = file.read()

            # 파일 메타데이터 생성
            metadata = {
                "file_path": file_path,
                "file_hash": calculate_file_hash(file_path)
            }

            # 벡터 스토어에 추가
            vector_store.add_texts([code_content], metadatas=[metadata])
            print(f"{file_path} 저장 완료!")
        except Exception as e:
            print(f"파일 처리 중 오류 발생: {file_path}, 오류: {e}")
    vector_store.persist()
    print("Chroma DB에 모든 파일 저장 완료!")

# Chroma DB에 저장된 모든 코드 파일의 메타데이터와 내용을 출력합니다.
def list_all_files_with_content(vector_store):
    try:
        results = vector_store.get(include=["metadatas", "documents"])

        print("저장된 코드 파일과 내용:")
        for i, (metadata, document) in enumerate(zip(results["metadatas"], results["documents"]), 1):
            file_path = metadata.get("file_path", "Unknown")
            print(f"{i}. 파일 경로: {file_path}")
            print("내용:")
            print(document[:300])  # 첫 300자만 출력
            print("-" * 50)
    except Exception as e:
        print(f"전체 파일 목록 및 내용 출력 중 오류 발생: {e}")

# 파일명을 기준으로 Chroma DB에서 내용을 검색하고 반환.
def search_by_filename(file_name, vector_store):
    try:
        # Chroma DB의 모든 데이터를 가져옵니다.
        results = vector_store.get(include=["metadatas", "documents"])

        matching_files = []
        for metadata, document in zip(results["metadatas"], results["documents"]):
            if file_name in metadata.get("file_path", ""):
                matching_files.append((metadata, document))

        if matching_files:
            return matching_files[0][1]
        else:
            print(f"파일명 '{file_name}'이(가) DB에 존재하지 않습니다.")
            return None
    except Exception as e:
        print(f"파일명으로 검색 중 오류 발생: {e}")

In [143]:
github_repo_url = 'https://github.com/adunStudio/codereviewexample'
local_repo_path = "./temp_repo"

# 1. GitHub 레포지토리 클론 또는 업데이트
clone_github_repo(github_repo_url, local_repo_path)

# 2. Python 파일 찾기
python_files = find_python_files(local_repo_path)
print(f"찾은 Python 파일 수: {len(python_files)}")

# 3. Python 파일을 Chroma DB에 저장
store_python_files_in_chroma(python_files, vector_store)

#list_all_files_with_content(vector_store)

로컬에 이미 레포지토리가 존재합니다. 변경 사항을 pull합니다.
레포지토리 클론 또는 업데이트 완료!
찾은 Python 파일 수: 8
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/world.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/cat.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/text_processor.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/bird.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/animal.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/calculator.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/dog.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/main.py
Chroma DB에 모든 파일 저장 완료!


---
# 6. Retriever

In [144]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [145]:
input_code = search_by_filename('cat.py', vector_store)

retrievered_codes = retriever.invoke(input_code)

print("\n### 출처 코드 ###")
print(f'문서 개수: {len(retrievered_codes)}')
for code in retrievered_codes:
    print(f"파일: {code.metadata.get('file_path', 'N/A')}")
    print(f"내용: {code.page_content[:200]}")  # 문서 내용 일부 출력
    print('-------' * 5)


### 출처 코드 ###
문서 개수: 5
파일: ./temp_repo/cat.py
내용: from animal import Animal

class Cat(Animal):
    def __init__(self, name, age, color):
        super().__init__(name, age)
        self.color = color
        self.sound = "야옹"

    def scratch(self, 
-----------------------------------
파일: ./temp_repo/dog.py
내용: from animal import Animal

class Dog(Animal):
    def __init__(self, name, age, breed):
        super().__init__(name, age)
        self.breed = breed
        self.sound = "멍멍"

    def fetch(self, it
-----------------------------------
파일: ./temp_repo/animal.py
내용: class Animal:
    def __init__(self, name, age):
        self.name = name
        self.age = age  
        self.sound = None

    def make_sound(self):
        print(f"{self.name}이(가) 소리를 냅니다: {self.s
-----------------------------------
파일: ./temp_repo/bird.py
내용: from animal import Animal

class Bird(Animal):
    def __init__(self, name, age, wing_span):
        super().__init__(name, age)
        self.wing_span =

---
# 7. PromptTemplates

In [146]:
from langchain.prompts import PromptTemplate
from langchain.prompts import load_prompt

review_template = load_prompt('./prompts/simple_review_template.json')

input_message = ''

for code in retrievered_codes:
    input_message += f"\n\n[파일명: {code.metadata.get('file_path', 'N/A')}]\n\n"
    input_message += code.page_content

review_prompt = review_template.format(context=input_message)
print(review_prompt)


<system>
당신은 동료를 위해 코드 리뷰를 수행하는 선임 소프트웨어 엔지니어입니다.
리뷰 항목입니다.
- 구글 코딩 스타일 가이드에 맞는지 꼼꼼히 봐주세요.
- 성능적으로 체크 (시간 복잡도, 공간 복잡도)
- 불필요한 코드가 있는지 봐주세요.
- 이미 내장된 기능에 있어 바퀴 재발명을 하고있는지 봐주세요.
- 확장 및 유지 보수성, SOLID 준수를 하고 있는지 봐주세요.
- DDD 법칙 준수를 선호합니다.
- 테스트 가능성 및 안정성도 봐주세요.
</system>

---

여러 파일과 함께 코드들이 주어집니다.
다음은 리뷰할 코드입니다:


[파일명: ./temp_repo/cat.py]

from animal import Animal

class Cat(Animal):
    def __init__(self, name, age, color):
        super().__init__(name, age)
        self.color = color
        self.sound = "야옹"

    def scratch(self, item):
        if not item:
            print(f"{self.name}이(가) 긁을 것이 없습니다!")
        else:
            print("{0}이(가) {1}을(를) 긁습니다.".format(self.name, self.item))

    def info(self):
        super().info()
        print("색깔: " + self.color)

[파일명: ./temp_repo/dog.py]

from animal import Animal

class Dog(Animal):
    def __init__(self, name, age, breed):
        super().__init__(name, age)
        self.breed = breed
        self.sound = "멍멍"

    def fet

---
# OpenAI 응답 결과

In [147]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as callback:
    simple_openai_response = openai_llm.invoke(review_prompt)

    # 토큰 관련 정보 출력
    print(f"입력 토큰 수: {callback.prompt_tokens}")
    print(f"응답 토큰 수: {callback.completion_tokens}")
    print(f"총 토큰 수: {callback.total_tokens}")
    print(f"총 비용: ${callback.total_cost}")

입력 토큰 수: 919
응답 토큰 수: 751
총 토큰 수: 1670
총 비용: $0.0005884499999999999


In [148]:
print(simple_openai_response.content)

[파일명: ./temp_repo/cat.py]

- `scratch` 메서드에서 `self.item` 대신 `item`을 사용해야 합니다. 현재 코드에서 `self.item`은 정의되지 않았습니다.
- 성능 및 메모리 사용 측면에서는 큰 문제가 없으나, 코딩 스타일에서 불필요하게 '야옹'이라는 문자열이 하드코딩 되어 있습니다. 

수정 제안:
```python
def scratch(self, item):
    if not item:
        print(f"{self.name}이(가) 긁을 것이 없습니다!")
    else:
        print(f"{self.name}이(가) {item}을(를) 긁습니다.")
```

[파일명: ./temp_repo/dog.py]

- `fetch` 메서드에서 불필요한 반복문이 있습니다. 현재 구현에서 1,000,000번의 빈 반복문은 성능에 큰 영향을 미칩니다.
- `fetch` 메서드를 호출하면 항상 `"공"`이라는 동일한 아이템을 사용하는 것은 하드코딩으로 비효율적입니다. 메서드 인자로 받는 것이 좋습니다.

수정 제안:
```python
def fetch(self, item):
    print(f"{self.name}이(가) {item}을(를) 가져옵니다!")
```

[파일명: ./temp_repo/animal.py]

- Animal 클래스는 잘 설계되었고 코드 스타일 가이드에 부합합니다. SOLID 원칙을 대체로 잘 지키고 있습니다. 다만, `make_sound`와 `info` 메서드에서 `sound`을 하드코딩하는 부분을 피하여 자식 클래스에서 오버라이드 가능하도록 변경하는 것이 좋습니다.

수정 제안:
```python
def make_sound(self):
    print(f"{self.name}이(가 소리를 냅니다: {self.sound if self.sound else '소리 없음'}")
```

[파일명: ./temp_repo/bird.py]

- Bird 클래스는 코딩 스타일을 잘 따르

---
# Anthropic 응답 결과

In [149]:
simple_anthropic_response = anthropic_llm.invoke(review_prompt)
print(simple_anthropic_response.content)

파일별 리뷰:

[./temp_repo/cat.py]
1. scratch 메서드에서 item 매개변수와 self.item 불일치 오류
```python
def scratch(self, item):
    print(f"{self.name}이(가) {item}을(를) 긁습니다.")  # self.item을 item으로 수정
```

[./temp_repo/dog.py]
1. fetch 메서드의 불필요한 for 루프 제거 필요
2. 시간 복잡도 O(n)을 O(1)로 개선
```python
def fetch(self, item):
    print(f"{self.name}이(가) {item}을(를) 가져옵니다!")
```

[./temp_repo/animal.py]
1. sound 속성의 기본값을 None 대신 빈 문자열로 설정 권장
```python
def __init__(self, name, age):
    self.name = name
    self.age = age
    self.sound = ""
```

[./temp_repo/bird.py]
1. 문자열 연산 최적화 필요
```python
def fly(self):
    print(f"{self.name}이(가) 날개 길이 {self.wing_span}cm로 날아오릅니다!")
```

[./temp_repo/world.py]
1. isinstance 체크 대신 다형성 활용 권장
2. 타입 힌트 추가 권장
```python
from typing import List
from animal import Animal

class World:
    def __init__(self):
        self.animals: List[Animal] = []

    def add_animal(self, animal: Animal) -> None:
        if isinstance(animal, Animal):
            self.animals.append(animal)
        else